In [16]:
import pandas as pd
import numpy as np
import pencilbox as pb
from requests import get
import requests

In [17]:
con_rs = pb.get_connection("[Warehouse] Redshift")

In [24]:
sql = """

with RMNGC as ( 
    Select dim_customer_key,device_id from (
                    Select 
                        dim_customer_key,
                        device_id,
                        city_name,
                        delivered_cart_rank,
                        cart_checkout_ts_ist,
                        row_number() over (partition by dim_customer_key order by cart_checkout_ts_ist desc) as rk 
                    from 
                        dwh.fact_sales_order_details
                    where 
                        cart_checkout_ts_ist::date  between '2022-03-01' and '2022-04-30'
                        and delivered_cart_rank >= 2 )
            where rk=1 
            and device_id!='00000000-0000-0000-0000-000000000000'
            and dim_customer_key not in ( Select dim_customer_key 
                                                from dwh.fact_sales_order_details 
                                                where cart_checkout_ts_ist::date  between '2022-03-01' and '2022-04-30'
                                                and delivered_cart_rank = 1 )                             --subtracting NU base
            and dim_customer_key not in ( Select dim_customer_key from (
                                                                        Select  
                                                                            customer_id as dim_customer_key,
                                                                            priority_score,
                                                                            row_number() over (partition by customer_id,priority_score order by updated_at desc) as rk  
                                                                            from consumer.customer_priority_score_cut3) 
                                                                    where rk=1 and priority_score in (7,8)) --subtracting GC base



),

user_table as
(Select dim_customer_key,min(cart_checkout_ts_ist) as fcheckout_dt
from dwh.fact_sales_order_details
where cart_checkout_ts_ist::date between '2022-03-01' and '2022-04-30'
and device_id is not null
group by 1),

order_table as (
Select 
dim_customer_key,
total_selling_price,
order_id,
cart_checkout_ts_ist checkout_dt
from dwh.fact_sales_order_details
where cart_checkout_ts_ist::date between '2022-03-01' and '2022-04-30'
and order_type in ('RetailForwardOrder', 'DropShippingForwardOrder')
and order_current_status = 'DELIVERED'
group by 1,2,3,4),

pre_final as (
Select 
nt.dim_customer_key,
sum(ot.total_selling_price) as total_gmv,
count(ot.total_selling_price) as total_orders
from 
user_table nt 
left join order_table ot
on nt.dim_customer_key=ot.dim_customer_key 
--and ot.checkout_dt::date between nt.fcheckout_dt::date and (nt.fcheckout_dt+7)::date
group by 1
),

final as (
select dim_customer_key
from
pre_final
where (total_gmv/total_orders) >300
and total_orders >1
group by 1),

dev as (
    Select 
        advertising_id,
        device_uuid 
    from spectrum.mobile_event_data where name in ('Cart Viewed') and at_date_ist >=current_date-360
 group by 1,2)
 
Select advertising_id as mobile_device_id from (
Select dev.advertising_id,
        case when RMNGC.dim_customer_key %% 41 < 4 then 'CG' else 'TG' end as cohort 
        from RMNGC inner join dev on RMNGC.device_id=dev.device_uuid
        inner join final on RMNGC.dim_customer_key=final.dim_customer_key
        
         ) 
         where advertising_id is not null
          and cohort='TG';

"""

In [25]:
df = pd.read_sql(sql=sql, con=con_rs)
df.rename(columns = {'mobile_device_id':'Mobile Device ID'}, inplace = True)
df.head()

,Mobile Device ID
0,228c52c1-3861-421a-9c9a-b35aeff90659
1,5c1a991f-c130-4bf6-ba72-29ce775faa8e
2,cca9a9da-ff3a-4094-93f5-31a96705de39
3,44651240-82ae-41b4-b826-c2c607bf73d3
4,97674789-8414-4b23-9676-a1acf330ed04


In [26]:
df.shape

(414726, 1)

In [27]:
df.to_csv('Repeat60-GC-NU-AOV300_LTO2.csv',index=False)